Takes the skimmed parquet files (output of bbVVSkimmer) and evaluates the HWW Tagger.

Author(s): Raghav Kansal

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
plt.rcParams.update({"font.size": 24})

import os
from os import listdir
from os.path import exists
import pickle
from pathlib import Path

from tqdm import tqdm

import utils, postprocessing, plotting
from HHbbVV.hh_vars import samples, nonres_samples, res_samples, nonres_sig_keys, res_sig_keys

MAIN_DIR = Path("../../../")

samples = samples | nonres_samples

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
plot_dir = MAIN_DIR / "plots/TaggerAnalysis/24May20"
# plot_dir = MAIN_DIR / "plots/BDT/24Apr9"
plot_dir.mkdir(parents=True, exist_ok=True)

samples_dir = f"{MAIN_DIR}/../data/skimmer/24Mar14UpdateData"
year = "2018"

Go down directly to BDT plots section for BDT plots

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
from collections import OrderedDict


res_samples = OrderedDict()

res_mps = [
    # (1000, 125),
    # (1400, 125),
    (1400, 150),
    # (1800, 125),
    (1800, 150),
    (1800, 190),
    # (2200, 125),
    (2200, 150),
    (2200, 190),
    (2200, 250),
    # (3000, 125),
    # (3000, 150),
    # (3000, 190),
    # (3000, 250),
    # (3000, 350),
]

for mX, mY in res_mps:
    res_samples[f"X[{mX}]->H(bb)Y[{mY}](WW)"] = (
        f"NMSSM_XToYH_MX{mX}_MY{mY}_HTo2bYTo2W_hadronicDecay"
    )

res_sig_keys = list(res_samples.keys())
nonres_sig_keys = ["HHbbVV"]

In [ ]:
# (column name, number of subcolumns)
load_columns = [
    ("weight", 1),
    ("weight_noTrigEffs", 1),
    ("ak8FatJetPt", 2),
    ("ak8FatJetParticleNetMass", 2),
    ("VVFatJetParTMD_THWWvsT", 1),
    ("VVFatJetParTMD_probHWW4q", 1),
    ("VVFatJetParTMD_probHWW3q", 1),
    ("VVFatJetParTMD_probQCD", 1),
    ("VVFatJetParTMD_probT", 1),
    # ("VVFatJetParticleNet_Th4q", 2),
]

# Both Jet's Regressed Mass above 50, electron veto
events_dict = postprocessing.load_samples(
    samples_dir,
    {key: samples[key] for key in nonres_sig_keys + ["QCD", "TT"]},
    year,
    filters=postprocessing.load_filters,
    columns=utils.format_columns(load_columns),
    variations=False,
)

In [ ]:
cutflow = pd.DataFrame(index=list(events_dict.keys()))
utils.add_to_cutflow(events_dict, "Preselection", "finalWeight", cutflow)
cutflow

### Cuts

In [ ]:
"""
``cuts_dict`` will be of format:
{
    sample1: {
        "cut1var1_min_max_cut1var2...": cut1,
        "cut2var2...": cut2,
        ...
    },
    sample2...
}
"""

pt_key = "Pt"
msd_key = "ParticleNetMass"
var_prefix = "ak8FatJet"

cutvars_dict = {"Pt": "pt", "Msd": "msoftdrop"}

all_cuts = [
    {pt_key: [300, 3000]},
    # {pt_key: [400, 600], msd_key: [60, 150]},
    # {pt_key: [300, 1500], msd_key: [110, 140]},
]

var_labels = {pt_key: r"$p_T$", msd_key: r"$m_{Reg}$"}

cuts_dict = {}
cut_labels = {}  # labels for plot titles, formatted as "var1label: [min, max] var2label..."

for sample, events in events_dict.items():
    # print(sample)
    cuts_dict[sample] = {}
    for cutvars in all_cuts:
        cutstrs = []
        cutlabel = []
        cuts = []
        for cutvar, (cutmin, cutmax) in cutvars.items():
            cutstrs.append(f"{cutvars_dict[cutvar]}_{cutmin}_{cutmax}")
            cutlabel.append(f"{var_labels[cutvar]}: [{cutmin}, {cutmax}]")
            cuts.append(events[f"{var_prefix}{cutvar}"] >= cutmin)
            cuts.append(events[f"{var_prefix}{cutvar}"] < cutmax)

        cutstr = "_".join(cutstrs)
        cut = np.prod(cuts, axis=0)
        cuts_dict[sample][cutstr] = cut.astype(bool)

        if cutstr not in cut_labels:
            cut_labels[cutstr] = " ".join(cutlabel)

### Histograms

In [ ]:
plot_vars = {
    # "th4q": {
    #     "title": "ParticleNet Non-MD Th4q",
    #     "score_label": "ak8FatJetParticleNet_Th4q",
    #     "colour": "orange",
    # },
    # "thvv4q": {
    #     "title": "ParT MD THVV",
    #     "score_label": "ak8FatJetParTMD_THWW4q",
    #     "colour": "green",
    # },
    "thvv4qt": {
        "title": r"GloParT $T_{HVV}$",
        "score_label": "VVFatJetParTMD_THWWvsT",
        "colour": "green",
    },
}

In [ ]:
samples = {"tt": "TT", "qcd": "QCD", "HHbbVV": "HHbbVV"}

plt.rcParams.update({"font.size": 16})
plt.style.use(hep.style.CMS)
hep.style.use("CMS")

for t, pvars in plot_vars.items():
    for cutstr in cut_labels:
        plt.figure(figsize=(16, 12))
        # plt.suptitle(f"HVV FatJet {pvars['title']} Scores", y=0.95)
        # plt.title(cut_labels[cutstr], fontsize=20)

        for sample, colour, skip in [("HHbbVV", "red", 1), ("QCD", "deepskyblue", 4)]:
            _ = plt.hist(
                events_dict[sample][pvars["score_label"]][cuts_dict[sample][cutstr]][::skip],
                histtype="step",
                bins=np.linspace(0, 1, 101),
                label=f"{sample}",
                linewidth=2,
                color=colour,
                density=True,
                weights=events_dict[sample]["weight"][cuts_dict[sample][cutstr]][::skip],
            )

        plt.ylabel("# Jets (A.U.)")
        plt.xlabel(f"{pvars['title']} Score")
        plt.legend()
        plt.savefig(
            f"{plot_dir}/{t}_hist_{cutstr}.pdf",
            bbox_inches="tight",
        )

### ROCs

In [ ]:
from sklearn.metrics import roc_curve, auc, integrate

rocs = {}
# sig_key = "HHbbVV"
bg_keys = ["TT", "QCD"]
bg_skip = 1


for cutstr in cut_labels:
    # print(cutstr)
    rocs[cutstr] = {}
    # for sig_key in tqdm(nonres_sig_keys + res_sig_keys):
    for sig_key in tqdm(nonres_sig_keys):
        rocs[cutstr][sig_key] = {}
        sig_cut = cuts_dict[sig_key][cutstr]
        bg_cuts = [cuts_dict[bg_key][cutstr] for bg_key in bg_keys]

        y_true = np.concatenate(
            [
                np.ones(np.sum(sig_cut)),
                np.zeros(int(np.ceil(np.sum(np.concatenate(bg_cuts)) / bg_skip))),
            ]
        )
        # print(y_true[np.sum(sig_cut):])

        weights = np.concatenate(
            [events_dict[sig_key]["weight"][sig_cut]]
            + [
                events_dict[bg_key]["weight"][bg_cut][::bg_skip]
                for bg_key, bg_cut in zip(bg_keys, bg_cuts)
            ],
        )

        for t, pvars in plot_vars.items():
            score_label = pvars["score_label"]
            scores = np.concatenate(
                [events_dict[sig_key][score_label][sig_cut]]
                + [
                    events_dict[bg_key][score_label][bg_cut][::bg_skip]
                    for bg_key, bg_cut in zip(bg_keys, bg_cuts)
                ],
            )
            # print(scores[np.sum(sig_cut):])
            fpr, tpr, thresholds = roc_curve(y_true, scores, sample_weight=weights)
            rocs[cutstr][sig_key][t] = {
                "fpr": fpr,
                "tpr": tpr,
                "thresholds": thresholds,
                # "auc": auc(fpr, tpr),
            }

All signals in same plot

In [ ]:
sig_split_points = [
    # [
    #     (1000, 125),
    #     (1400, 125),
    #     (1800, 125),
    #     (2200, 125),
    #     (3000, 125),
    # ],
    [
        (1400, 150),
        (1800, 150),
        (1800, 190),
        (2200, 150),
        (2200, 190),
        (2200, 250),
    ],
    # [
    #     (3000, 125),
    #     (3000, 150),
    #     (3000, 190),
    #     (3000, 250),
    #     (3000, 350),
    # ],
]

sig_splits = [
    ["HHbbVV"]  # + [f"X[{mX}]->H(bb)Y[{mY}](WW)" for (mX, mY) in mps] for mps in sig_split_points
]

In [ ]:
roc = rocs[cutstr][sig_key][t]
roc_auc = integrate.trapz(y=roc["tpr"], x=roc["fpr"])
print("AUC:", roc_auc)
plotting.rocCurve(roc["fpr"], roc["tpr"], show=True, plot_dir=plot_dir, name="THVV", auc=roc_auc)

In [ ]:
xlim = [0, 0.8]
ylim = [1e-6, 1]

plot_thresholds = [0.98, 0.96, 0.94, 0.9, 0.8, 0.6, 0.4]
th_colours = ["#9381FF", "#1f78b4", "#a6cee3", "#ff7f00", "#7CB518", "#EDB458", "#36213E"]

plt.rcParams.update({"font.size": 24})

sig_colours = [
    "#23CE6B",
    "#ffbaba",
    "#ff7b7b",
    "#ff5252",
    # "#EDB458",
    "#a70000",
    "#885053",
    "#3C0919",
]

plot_vars = {
    # "th4q": {
    #     "title": "ParticleNet Non-MD Th4q",
    #     "score_label": "ak8FatJetParticleNet_Th4q",
    #     "colour": "orange",
    # },
    # "thvv4q": {
    #     "title": "ParT MD THWW",
    #     "score_label": "ak8FatJetParTMD_THWW4q",
    #     "colour": "green",
    # },
    "thvv4qt": {
        "title": "ParT MD THVV",
        "score_label": "ak8FatJetParTMD_THWWvsT",
        "colour": "green",
    },
}


for cutstr in cut_labels:
    for t, pvars in plot_vars.items():
        for j, plot_sig_keys in enumerate(sig_splits):
            split_str = "allsigs" if len(sig_splits) == 1 else f"sigs{j}"
            pths = {th: [[], []] for th in plot_thresholds}
            plt.figure(figsize=(12, 12))
            for i, sig_key in enumerate(plot_sig_keys):
                roc = rocs[cutstr][sig_key][t]
                c = sig_colours[i]
                plt.plot(
                    roc["tpr"],
                    roc["fpr"],
                    # label=f"{sig_key} AUC: {roc['auc']:.2f}",
                    label=f"{sig_key}",
                    linewidth=2,
                    color=c,
                )
                for th in plot_thresholds:
                    idx = find_nearest(roc["thresholds"], th)
                    pths[th][0].append(roc["tpr"][idx])
                    pths[th][1].append(roc["fpr"][idx])

                # plt.vlines(
                #     x=roc["tpr"][np.searchsorted(roc["fpr"], 0.01)],
                #     ymin=0,
                #     ymax=0.01,
                #     colors=c,
                #     linestyles="dashed",
                # )

            for k, th in enumerate(plot_thresholds):
                plt.scatter(
                    *pths[th],
                    marker="o",
                    s=40,
                    label=f"THVV > {th:.2f}",
                    color=th_colours[k],
                    zorder=100,
                )
            hep.cms.label(data=False, rlabel="")
            # plt.hlines(y=0.01, xmin=0, xmax=1, colors="lightgrey", linestyles="dashed")
            plt.yscale("log")
            plt.xlabel("Signal efficiency")
            plt.ylabel("Background efficiency")
            # plt.suptitle(f"HVV FatJet {pvars['title']} ROC", y=0.95)
            # plt.title(cut_labels[cutstr], fontsize=20)
            plt.xlim(*xlim)
            plt.ylim(*ylim)
            plt.legend(loc="lower right")
            plt.savefig(f"{plot_dir}/roccurve_{split_str}_{t}_{cutstr}.pdf", bbox_inches="tight")

Old vs New Tagger in same plot

In [ ]:
xlim = [0, 0.6]
ylim = [1e-6, 1]

for cutstr in cut_labels:
    plt.figure(figsize=(12, 12))
    for t, pvars in plot_vars.items():
        plt.plot(
            rocs[cutstr][t]["tpr"][::10],
            rocs[cutstr][t]["fpr"][::10],
            label=f"{pvars['title']} AUC: {rocs[cutstr][t]['auc']:.2f}",
            linewidth=2,
            color=pvars["colour"],
        )
        plt.vlines(
            x=rocs[cutstr][t]["tpr"][np.searchsorted(rocs[cutstr][t]["fpr"], 0.01)],
            ymin=0,
            ymax=0.01,
            colors=pvars["colour"],
            linestyles="dashed",
        )
    plt.hlines(y=0.01, xmin=0, xmax=1, colors="lightgrey", linestyles="dashed")
    plt.yscale("log")
    plt.xlabel("Signal Eff.")
    plt.ylabel("BG Eff.")
    plt.suptitle("HVV FatJet ROC Curves", y=0.95)
    plt.title(cut_labels[cutstr], fontsize=20)
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    plt.legend()
    plt.savefig(f"{plot_dir}/roccurve_{cutstr}.pdf", bbox_inches="tight")

### BDT Plots

Checking thresholds

In [ ]:
with open(f"{samples_dir}/24_04_05_k2v0_training_eqsig_vbf_vars_rm_deta/roc_dict.pkl", "rb") as f:
    roc = pickle.load(f)

In [ ]:
plotting.multiROCCurve(
    {"test": roc},
    thresholds=[[0.98, 0.99, 0.994, 0.995, 0.9965], [0.99, 0.997, 0.998, 0.999, 0.9997]],
    plot_dir=plot_dir,
    name="roc_thresholds",
    show=True,
)

In [ ]:
xlim = [0, 0.3]
ylim = [1e-6, 1e-2]

# plot_thresholds = [0.6, 0.9, 0.96, 0.99, 0.997, 0.998, 0.999]
plot_thresholds = [0.997, 0.998, 0.999, 0.9993, 0.9995, 0.9997, 0.9999]
th_colours = [
    "#36213E",
    "#9381FF",
    "#1f78b4",
    # "#a6cee3",
    # "#32965D",
    "#7CB518",
    "#EDB458",
    "#ff7f00",
    "#a70000",
]

roc_colours = [
    "#23CE6B",
    "#a70000",
]

plt.rcParams.update({"font.size": 24})

pths = {th: [[], []] for th in plot_thresholds}
plt.figure(figsize=(12, 12))
c = roc_colours[0]

plt.plot(
    roc["tpr"],
    roc["fpr"],
    # label=roc_labels[inf],
    linewidth=2,
    color=c,
)

for th in plot_thresholds:
    idx = find_nearest(roc["thresholds"], th)
    pths[th][0].append(roc["tpr"][idx])
    pths[th][1].append(roc["fpr"][idx])

for k, th in enumerate(plot_thresholds):
    plt.scatter(
        *pths[th],
        marker="o",
        s=40,
        label=f"BDT Score > {th}",
        color=th_colours[k],
        zorder=100,
    )

hep.cms.label(data=False, rlabel="")
plt.yscale("log")
plt.xlabel("Signal efficiency")
plt.ylabel("Background efficiency")
plt.xlim(*xlim)
plt.ylim(*ylim)
plt.legend(loc="lower right")
# plt.savefig(f"{plot_dir}/roc_bdt.pdf", bbox_inches="tight")
plt.show()

Comparing binary vs multi-class

In [ ]:
inferences_dir = f"{samples_dir}/inferences/"
inferences_dir_binary = f"{samples_dir}/inferences_binary/"
rocs = {"inferences": {}, "inferences_binary": {}}

for arr in ["tpr", "fpr", "thresholds"]:
    print(arr)
    rocs["inferences"][arr] = np.loadtxt(f"{inferences_dir}/{arr}.txt")
    # rocs["inferences_binary"][arr] = np.loadtxt(f"{inferences_dir_binary}/{arr}.txt")

roc_labels = {"inferences": "Multiclass", "inferences_binary": "Binary"}

In [ ]:
xlim = [0, 1]
ylim = [1e-6, 1]

roc_colours = [
    "#23CE6B",
    "#a70000",
]

plt.rcParams.update({"font.size": 24})

pths = {th: [[], []] for th in plot_thresholds}
plt.figure(figsize=(12, 12))
for i, inf in enumerate(["inferences", "inferences_binary"]):
    roc = rocs[inf]
    c = roc_colours[i]

    plt.plot(
        roc["tpr"],
        roc["fpr"],
        label=roc_labels[inf],
        linewidth=2,
        color=c,
    )

hep.cms.label(data=False, rlabel="")
plt.yscale("log")
plt.xlabel("Signal efficiency")
plt.ylabel("Background efficiency")
plt.xlim(*xlim)
plt.ylim(*ylim)
plt.legend(loc="upper left")
plt.savefig(f"{plot_dir}/roc_multi_binary_bdt.pdf", bbox_inches="tight")

Comparing train vs test

In [ ]:
inferences_dir = f"{samples_dir}/inferences/"
rocs = {}

for inf in ["train", "test"]:
    with open(f"{inferences_dir}/23_11_02_{inf}_roc_dict.pkl", "rb") as f:
        rocs[inf] = pickle.load(f)

roc_labels = {"train": "Train", "test": "Test"}

In [ ]:
xlim = [0, 1]
ylim = [1e-6, 1]

plot_thresholds = [0.6, 0.9, 0.96, 0.99, 0.997, 0.998, 0.999]
th_colours = [
    "#36213E",
    "#9381FF",
    "#1f78b4",
    # "#a6cee3",
    # "#32965D",
    "#7CB518",
    "#EDB458",
    "#ff7f00",
    "#a70000",
]

roc_colours = [
    "#23CE6B",
    "#a70000",
]

plt.rcParams.update({"font.size": 24})

pths = {th: [[], []] for th in plot_thresholds}
plt.figure(figsize=(12, 12))
for i, inf in enumerate(["inferences"]):
    roc = rocs[inf]
    c = roc_colours[i]

    plt.plot(
        roc["tpr"],
        roc["fpr"],
        # label=roc_labels[inf],
        linewidth=2,
        color=c,
    )

    for th in plot_thresholds:
        idx = find_nearest(roc["thresholds"], th)
        pths[th][0].append(roc["tpr"][idx])
        pths[th][1].append(roc["fpr"][idx])

for k, th in enumerate(plot_thresholds):
    plt.scatter(
        *pths[th],
        marker="o",
        s=40,
        label=f"BDT Score > {th}",
        color=th_colours[k],
        zorder=100,
    )

hep.cms.label(data=False, rlabel="")
plt.yscale("log")
plt.xlabel("Signal efficiency")
plt.ylabel("Background efficiency")
plt.xlim(*xlim)
plt.ylim(*ylim)
plt.legend(loc="lower right")
# plt.savefig(f"{plot_dir}/roc_bdt.pdf", bbox_inches="tight")

Trimming features

In [ ]:
fpr_tpr_th = np.array(
    [
        6.841912584548129e-05,
        0.06309407980527611,
        0.997972309589386,
        5.5970691547507736e-05,
        0.06309112513484341,
        0.9981555342674255,
        6.062037416775686e-05,
        0.06309433205678786,
        0.9980131387710571,
        6.282697172598808e-05,
        0.06309181657972177,
        0.9981679916381836,
        7.333505621545634e-05,
        0.06308829354373516,
        0.9979708790779114,
        7.116201530873543e-05,
        0.06309218251750907,
        0.9979798197746277,
        7.2475727068077e-05,
        0.06310060756610338,
        0.9979470372200012,
        7.738615489911652e-05,
        0.06309448855598651,
        0.9979099631309509,
        8.047411596104979e-05,
        0.06309097845150874,
        0.9977815747261047,
        8.503279607366034e-05,
        0.06308554741097783,
        0.9978626370429993,
        8.495399767538146e-05,
        0.06309224241230124,
        0.9976242184638977,
        8.807786735948429e-05,
        0.06308749819238176,
        0.9974913597106934,
        0.00034950987699599445,
        0.06309245484995998,
        0.9928722381591797,
    ]
)

fpr = fpr_tpr_th[::3]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(fpr)
plt.xlabel("Features Removed")
plt.ylabel("Background efficiency")
plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.savefig(f"{plot_dir}/feats_removed.pdf", bbox_inches="tight")

In [ ]:
uls = np.array(
    [[0, 53.25], [1, 50.25], [3, 49.5], [4, 49.9375], [6, 50.25], [8, 52.75], [11, 58.8]]
)

plt.figure(figsize=(12, 8))
plt.plot(uls[:, 0], uls[:, 1])
plt.xlabel("Features Removed")
plt.ylabel("Median expected upper limit xSM")
# plt.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
plt.savefig(f"{plot_dir}/feats_removed_uls.pdf", bbox_inches="tight")